# Example 07 - Multicollinearity
This example will use different data that is presented in the textbook.    
I could not find the book by (Johnson and Wichern, 1997) - *Business statistics - Decision making with data*.  

Therefore, I replaced the data with the data from (Donnelly, 2020) - *Business statistics*. Figure 15.15A

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tabulate
import scipy.stats as sc_stats

from IPython.display import display

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline


In [3]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
  'font.size': 12,
  'grid.alpha': 0.25})

## Function declarations



### Read `ods` to `DataFrame`

In [4]:
def read_ods(filename, columns=None):
  if columns is None:
    df = pd.read_excel(filename, engine="odf", header=0)
  elif isinstance(columns, list):
    df = pd.read_excel(filename, engine="odf", header=0,
                       usecols=columns)
  else:
    key_arr = (k for k in columns.keys())
    df = pd.read_excel(filename, engine="odf", header=0,
                      usecols=key_arr)
    df = df.rename(columns=columns)
  return df

### Get multiple regression coefficients

In [5]:
def get_multiple_regress_coeff(df, n_vars=2):
  
  if isinstance(df, pd.DataFrame):
    num_of_samples = len(df)
    X_arr = np.zeros((num_of_samples, n_vars))
    Y = df["Y"].to_numpy()

    for i in range(n_vars):
      key = f"X_{i+1}"
      X_arr[:, i] = df[key].to_numpy()

    regress = LinearRegression().fit(X_arr, Y)
    df["hat_Y"] = regress.intercept_ + X_arr.dot(regress.coef_) 
  
  elif isinstance(df, np.ndarray):
    Y = df[:, 0]
    X_arr = df[:, 1:]


    regress = LinearRegression().fit(X_arr, Y)
    Y_hat = regress.intercept_ + X_arr.dot(regress.coef_)
    df = np.column_stack([df, Y_hat])

  return regress, df


### Get sum of square of $[Y, \mathbf{X}]$

In [6]:
def get_sumSq(data_struct, n_vars=2):
  regress, _ = get_multiple_regress_coeff(data_struct, n_vars=n_vars)

  hat_Y =  regress.intercept_ + data_struct[:,1:].dot(regress.coef_)
  
  meanY = data_struct[:, 0].mean()
  SST = ((data_struct[:, 0] - meanY)**2).sum()
  SSR = ((hat_Y - meanY)**2).sum()
  SSE = ((data_struct[:, 0] - hat_Y)**2).sum()

  return SST, SSR, SSE

### Get Minitab output of Regression Analysis

In [7]:
def get_minitab_out(df, n_vars=2, new_column=None, withVIF=False):
  """
  new_column (dict) : a dictionary mapping to rename df header
  """
  
  data_struct = np.zeros((len(df), 1+n_vars))
  data_struct[:, 0] = df['Y']
  for i in range(n_vars):
    key = f"X_{i+1}"
    data_struct[:, i+1] = df[key]

  # -- compute correlation matrix
  corr_matrix = np.corrcoef(data_struct, rowvar=False)
  corr_matrix = corr_matrix[1:, 0:-1]    # only show lower triangle part of correlation matrix

  # -- compute regression intercept and coefficients
  regress, _ = get_multiple_regress_coeff(df, n_vars=n_vars)
  b_arr = [regress.intercept_] + regress.coef_.tolist()
  is_positive_b_arr = [b_j > 0 for b_j in b_arr]

  # -- compute cofactor matrix 
  X_arr = np.ones_like(data_struct)
  X_arr[:,1:] = data_struct[:,1:]
  cofactor_matrix = np.linalg.inv(X_arr.transpose().dot(X_arr))   # this matrix is closely related to covariance matrix

  # -- compute standad error of the estimates
  num_of_samples = len(df)
  hat_Y =  regress.intercept_ + data_struct[:,1:].dot(regress.coef_)
  sumSq_Y_hat_Y = ((data_struct[:, 0] - hat_Y)**2).sum()
  s_yxs = np.sqrt(sumSq_Y_hat_Y/(num_of_samples - n_vars - 1))

  # -- compute standard error of intercept_ and coef_
  # -- compute t-score of intecept_ and coef_
  # -- compute p-values of intercept_ and coef_
  SE_coef = np.zeros(1+n_vars)
  t_scores = np.zeros(1+n_vars)
  p_values = np.zeros(1+n_vars)
  dof = num_of_samples - 2;          # degrees of freedom
  for i in range(1+n_vars):
    SE_coef[i] = s_yxs*np.sqrt(cofactor_matrix[i, i])
    t_scores[i] = b_arr[i] / SE_coef[i]
    p_values[i] = sc_stats.t.pdf(t_scores[i], dof)

  # -- compute VIF of coef_
  if withVIF:
    # if n_vars == 2, we calculate VIF from correlation matrix
    VIF_arr = np.zeros(n_vars)
    if n_vars == 2:
      VIF_arr[0] = 1/(1-corr_matrix[1, 1]**2)
      VIF_arr[1] = 1/(1-corr_matrix[1, 1]**2)
    else:
      for i in range(n_vars):
        # take a predictor column and put the other predictor after it
        predictor_data_struct = np.zeros((num_of_samples, n_vars))
        predictor_data_struct[:,0] = data_struct[:, i+1]   
        predictor_data_struct[:,1:] = np.delete(data_struct, i+1, axis=1)[:,1:]  
        #print(predictor_data_struct)
        predictor_SST, predictor_SSR, _ = get_sumSq(predictor_data_struct, n_vars=n_vars)
        predictor_R_sq = predictor_SSR/predictor_SST
        #print(predictor_R_sq)
        VIF_arr[i] = 1/(1 - predictor_R_sq)


  # -- compute ANOVA table
  meanY = data_struct[:, 0].mean()
  SST = ((data_struct[:, 0] - meanY)**2).sum()
  SSR = ((hat_Y - meanY)**2).sum()
  SSE = ((data_struct[:, 0] - hat_Y)**2).sum()
  anova_sumSq = np.array([SSR, SSE, SST])
  anova_dof = np.array([n_vars, num_of_samples - n_vars - 1, num_of_samples - 1], dtype=int)
  anova_meanSq = (anova_sumSq/anova_dof)[:2]
  anova_F_score = anova_meanSq[0] / anova_meanSq[1]
  anova_p_value = sc_stats.f.pdf(anova_F_score, anova_dof[0], anova_dof[1])

  # -- compute R_sq (coefficient of determination)
  R_sq = SSR/SST

  # -- compute adjusted R_sq
  adj_R_sq = 1 - (1 - R_sq)*(num_of_samples-1)/(num_of_samples-n_vars-1)

  # -- create tabular form for correlation 
  data = []
  for i in range(n_vars):
    data_row = [new_column[i+1], *corr_matrix[i,:]]
    data.append(data_row)

  table_corr = tabulate.tabulate(data, tablefmt='html', 
    headers=[""] + new_column[:n_vars], 
    floatfmt=[None] + [".2f"]*n_vars)

  print(f"Correlations: {', '.join(new_column)}")
  display(table_corr)

  # -- create tabular form for predictor
  data = [["Constant", b_arr[0], SE_coef[0], t_scores[0], p_values[0]]]
  if withVIF:
    for i in range(1, n_vars+1):
      data_row = [new_column[i], b_arr[i], SE_coef[i], t_scores[i], p_values[i], VIF_arr[i-1]]
      data.append(data_row)
    table_predictor = tabulate.tabulate(data, tablefmt='html', 
      headers=["Predictor", "Coef", "SE Coef", "t-score", "p-value", "VIF"], 
      floatfmt=[None, ".4f", ".4f", ".2f", ".3f", ".3f"])
  else:
    for i in range(1, n_vars+1):
      data_row = [new_column[i], b_arr[i], SE_coef[i], t_scores[i], p_values[i]]
      data.append(data_row)
    table_predictor = tabulate.tabulate(data, tablefmt='html', 
      headers=["Predictor", "Coef", "SE Coef", "t-score", "p-value"], 
      floatfmt=[None, ".4f", ".4f", ".2f", ".3f"])


  print(f"Regression Analysis: {new_column[0]} versus {', '.join(new_column[1:])}")
  print("The regression equation is")
  str_b_predictor = "" 
  for i in range(n_vars):
    sign = "+" if is_positive_b_arr[i+1] else "-"
    str_b_predictor += f" {sign} {abs(b_arr[i+1]):.3f} {new_column[i+1]}"
  print(f"{new_column[0]} = {b_arr[0]:.3f}{str_b_predictor}")
  display(table_predictor)
  print(f"s_yxs = {s_yxs:.4f}   R_sq = {R_sq*100:.1f}%   R-sq(adj) = {adj_R_sq*100:.1f}%")

  # -- create ANOVA table (with F-score and its p-value)
  data = [
    ["Regression",     anova_dof[0], anova_sumSq[0], f"{anova_meanSq[0]:.3f}", f"{anova_F_score:.3f}", f"{anova_p_value:.3f}"],
    ["Residual error", anova_dof[1], anova_sumSq[1], f"{anova_meanSq[1]:.3f}", "", ""],
    ["Total",          anova_dof[2], anova_sumSq[2], "", "", ""]] 
  table_anova = tabulate.tabulate(data, tablefmt='html', 
    headers=["Source", "d.o.f", "sumSq", "meanSq", "F-score", "p-value"], 
    floatfmt=[None, ".0f", ".3f", "s", "s", "s"])

  print(f"Analyis of Variance")
  display(table_anova)

  return {"regress": regress, "corr_matrix": corr_matrix, 
    "R_sq": R_sq}


## Load the data

In [11]:
# For testing

# filename = "black-2009-14-table-06-crude-oil-production.ods"
# -- Full table
# df = read_ods(filename, columns=[
#   "World Crude Oil Production (million barrels per day)", 
#   "U.S. Energy Consumption (quadrillion BTUs generation per year)", 
#   "U.S. Nuclear Electricity (billion kilowatt-hours)", 
#   "U.S. Coal Gross Production (million short-tons)", 
#   "U.S. Total Dry Gas Production (trillion cubic feet)", 
#   "U.S. Fuel Rate for Automobiles (miles per gallon)"])

# filename = "donnelly-2020-15-your-turn-05-mba-gpa.ods"
# -- Full table
# df = read_ods(filename, columns=[
#   "MBAGPA", "MGMAT", "VGMAT", "UGGPA", "DEGREE"])
# -- Select two predictor variables with high VIF and one with close to one
# df = read_ods(filename, columns=[
#   "MBAGPA", "MGMAT", "VGMAT", "UGGPA"])

# filename = "donnelly-2020-15-problems-57-laptops.ods"
# -- Full table
# df = read_ods(filename, columns=[
#   "Price ($)", "Screen Size (in.)", "RAM Memory (GB)", 
#   "Hard drive (GB)", "USB Ports", "Weight (oz.)"])
# df = read_ods(filename, columns=[
#   "Price ($)", "RAM Memory (GB)", "USB Ports", "Weight (oz.)"])

# filename = "black-2009-14-case-discussion-01-size-purchase.ods"
# # -- Full table
# df = read_ods(filename, columns=[
#   "Size of Purchase ($1,000)", "Company Size ($ million sales)",
#   "Percent of Customer Imports", "Distance from Virginia Semiconductor"])

# filename = "neter-1983-11-table-02-body-fat.ods"
# -- Full table
# df = read_ods(filename, columns=[
#   "Body Fat Y_i",
#   "Triceps Skinfold Thickness X_i1",
#   "Thigh Circumference X_i2",
#   "Midarm Circumference X_i3"])

In [17]:
filename = "donnelly-2020-15-figure-15a-cell-phone-activation.ods"

# -- Full table
# df = read_ods(filename, columns=[
#   "Activations", "Exp", "Gend", "Perf", "Age", "Salary ($)"])

# -- Select two predictor variables with high VIF and one with close to one
df = read_ods(filename, columns=[
  "Activations", "Perf", "Age", "Salary ($)"])


df.head()


,Activations,Perf,Age,Salary ($)
0,19,80,27,810
1,20,76,32,720
2,20,82,46,960
3,22,82,35,750
4,23,80,41,850


## Calculate Minitab's output with VIF (3 predictors)

In [18]:
df_minitab = df.copy()
from_column_names = df_minitab.columns.to_list()
print(f"header: {from_column_names}")
to_column_names = ["Y"] + [f"X_{i+1}" for i in range(len(from_column_names) - 1)]


df_minitab = df_minitab.rename(columns=
  {k: v for k, v in zip(from_column_names, to_column_names)})

df_minitab.head()

header: ['Activations', 'Perf', 'Age', 'Salary ($)']


,Y,X_1,X_2,X_3
0,19,80,27,810
1,20,76,32,720
2,20,82,46,960
3,22,82,35,750
4,23,80,41,850


In [20]:
column_name = ["Activations", "Perf", "Age", "Salary"]
out_dict = get_minitab_out(df_minitab, n_vars=len(column_name)-1, 
  new_column=column_name, withVIF=True)

Correlations: Activations, Perf, Age, Salary


,Activations,Perf,Age
Perf,0.67,1.00,0.31
Age,0.06,0.31,1.00
Salary,0.76,0.89,0.21


Regression Analysis: Activations versus Perf, Age, Salary
The regression equation is
Activations = 3.117 + 0.046 Perf - 0.089 Age + 0.028 Salary


Predictor,Coef,SE Coef,t-score,p-value,VIF
Constant,3.1167,11.9997,0.26,0.381,
Perf,0.0463,0.2569,0.18,0.388,5.192
Age,-0.0892,0.1199,-0.74,0.297,1.134
Salary,0.0278,0.0118,2.37,0.029,4.904


s_yxs = 4.3753   R_sq = 59.0%   R-sq(adj) = 53.1%
Analyis of Variance


Source,d.o.f,sumSq,meanSq,F-score,p-value
Regression,3,578.560,192.853,10.074,0.000
Residual error,21,402.000,19.143,,
Total,24,980.560,,,


The $F$ statistic ($10.98$) and its $p$-value ($.000$) clearly indicate 
that the regression is significant. 
The $t$ statistic for `Perf` is small with a relatively large $p$-value. 
It must be concluded, that the variable `Perf` is not significant, 
provided the other predictor variables remain in th regression function.
This suggest that the term $\beta_1 X_1$ can be dropped from the regression 
function if the remaining terms, $\beta_2 X_2$ and $\beta_3 X_3$, are retained.
Similarly, it appears as if $\beta_2 X_2$ can be dropped if $\beta_1 X_1$
and $\beta_3 X_3$ remain in the regression function. The $t$ value ($2.37$)
associated with `Salary` is marginally significant.

Since $\textrm{VIF} = 1.1$ for `Age`, this predictor variable is very weakly related
($\textrm{VIF}$ near $1$) to the remaining predictor variables, `Perf` and `Salary`.  
The $\textrm{VIF} = 5.2$ for `Perf` is relatively large, indicating this variable
is linearly related to the remaining predictor variables. 
Also, the $\textrm{VIF} = 4.9$ for `Salary` indicates that `Salary` is 
related to the remaining predictor variables. Since `Age` is weakly related
to `Perf` and `Salary`, the relationship among the predictor
variables is essentially the relationship between `Perf` and `Salary`.
In fact, the sample correlation between `Perf` and `Salary` is 
$r = 0.89$, showing strong linear association.

Because of the strong linear relation between `Perf` and `Salary`. 
Those variables are very similar in their ability to explain `Activations`.
We need only one, but not both, in the regression function. The
Minitab output from a regression analysis with `Perf` 
(smallest absolute value of $t$ statistic)
deleted from the regression function is shown in the below cell output.

## Calculate Minitab's output with VIF (2 predictors)

In [21]:
df_minitab_2predictors = df.copy()
df_minitab_2predictors = df_minitab_2predictors.drop(columns=["Perf"])

from_column_names = df_minitab_2predictors.columns.to_list()
print(f"header: {from_column_names}")
to_column_names = ["Y"] + [f"X_{i+1}" for i in range(len(from_column_names) - 1)]


df_minitab_2predictors = df_minitab_2predictors.rename(columns=
  {k: v for k, v in zip(from_column_names, to_column_names)})

df_minitab_2predictors.head()


header: ['Activations', 'Age', 'Salary ($)']


,Y,X_1,X_2
0,19,27,810
1,20,32,720
2,20,46,960
3,22,35,750
4,23,41,850


In [23]:
column_name = ["Activations", "Age", "Salary"] 
out_dict = get_minitab_out(df_minitab_2predictors, 
  n_vars=len(column_name) - 1, new_column=column_name, withVIF=True)


Correlations: Activations, Age, Salary


,Activations,Age
Age,0.06,1.00
Salary,0.76,0.21


Regression Analysis: Activations versus Age, Salary
The regression equation is
Activations = 5.006 - 0.083 Age + 0.030 Salary


Predictor,Coef,SE Coef,t-score,p-value,VIF
Constant,5.0065,5.6963,0.88,0.265,
Age,-0.0832,0.1125,-0.74,0.298,1.044
Salary,0.0297,0.0053,5.60,0.000,1.044


s_yxs = 4.2780   R_sq = 58.9%   R-sq(adj) = 55.2%
Analyis of Variance


Source,d.o.f,sumSq,meanSq,F-score,p-value
Regression,2,577.939,288.969,15.790,0.000
Residual error,22,402.621,18.301,,
Total,24,980.560,,,


Notice that the coefficient of `Age` and `Salary` is about the same for the 
two regressions. 
Also, for the second regression, the variable `Salary` is
clearly significant ($t = 5.60$ with $p$-value $=.000$).
With `Age` in the model, `Salary` is an additional important
predictor of `Activations`. The $R^2$'s for the two regressions
are nearly the same, approximately $.59$, as are the standard
error of the estimates, $s_{y\cdot x's} = 4.375$ and 
$s_{y\cdot x's} = 4.278$, respectively. Finally, the common
$\textrm{VIF} = 1.0$ for the two predictors in the second model
indicates that multicollinearity is no longer a problem.